The goal of this notebook is to determine how well a simple spherical NFW potential with a static MW can approximate the full N-body potential of the LMC (for short term evolution).
In the long-term, whether or not the LMC remains bound depends on the approximation parameters (see other notebook here), but if we care about the effects of the LMC on Scl in the past ~0.2 Gyr, then this may not matter as much.

In [ ]:
using CairoMakie 
using Arya

using LilGuys

In [ ]:
using PythonCall
agama = pyimport("agama")

In [ ]:
np = pyimport("numpy")

In [ ]:
include("../../../notebooks/orbits/agama_utils.jl")

In [ ]:
potential_dir = ENV["DWARFS_ROOT"] * "/agama/potentials/"

In [ ]:
readdir(potential_dir )

# loading potential files

In [ ]:
scl_profile = NFW(r_circ_max=1.84, v_circ_max=25.75/V2KMS)

In [ ]:
pot_lmc_2 = agama.Potential("$potential_dir/vasiliev24/L2M10/pot/lmc000.ini")

In [ ]:
pot_lmc_3 = agama.Potential("$potential_dir/vasiliev24/L3M10/pot/lmc000.ini")

In [ ]:
pot_lmc_1_5 = agama.Potential("$potential_dir/vasiliev+21/lmc00.pot")

In [ ]:
pot_MW_9_5 = agama.Potential("$potential_dir/vasiliev+21/potential_nolmc.ini")

In [ ]:
pot_MW_10 = agama.Potential("$potential_dir/vasiliev24/L2M10/potential_mw_init.ini")

In [ ]:
pot_MW_11 = agama.Potential("$potential_dir/vasiliev24/L2M11/potential_mw_init.ini")

In [ ]:
pot_EP = agama.Potential("$potential_dir/EP2020.ini")

In [ ]:
function plot_acc!(ax, prof::Py; xlimits=(-2, 1.5), vasiliev_units=false, kwargs...)
    x = LinRange(xlimits[1], xlimits[2], 1000)
    r = np.array((10 .^ x) .* [0, 0, 1]')
    rho = prof.force(r)
    acc = pyconvert(Matrix{Float64}, rho)'
    a = calc_r(acc)
    r = 10 .^ x
    
    if vasiliev_units
        a .*= (V_M2MSUN ./ M2MSUN) ./ (V_T2GYR ./ T2GYR) .^ 2
    end

    lines!(x, log10.(a ./ r); kwargs...)
end